In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, ReLU, BatchNormalization, \
    Flatten, Dense, Reshape, Conv1DTranspose, Activation
from tensorflow.keras import backend as K
import numpy as np

import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from nltk.tokenize import word_tokenize
from keras_preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import keras.backend as K
from keras.layers import Layer
import gensim

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings(action = 'ignore')


## Data

In [ ]:
#Preprocessing function
def preprocessing(data_frame):
    ## Preprocessing
    # Removing URLs whithin the tweets
    data_frame["Text"] = data_frame["Text"].str.replace(r'\s*https?://\S+(\s+|$)', ' ').str.strip()
    # Removing emails, hashtags and punctuations
    data_frame['Text'] = data_frame["Text"].str.replace(r'\S*@\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'#\S*\s?', ' ').str.strip()
    data_frame['Text'] = data_frame['Text'].str.replace(r'[^\w\s]+', ' ').str.strip()

    # Lowercase Text
    data_frame['Text'] = data_frame['Text'].str.lower()

    # # Removing stopwords
    stop = stopwords.words('english')
    data_frame['Text'].apply(lambda x: [item for item in str(x) if item not in stop])

    # Removing newline characters
    data_frame['Text'] = data_frame['Text'].str.rstrip()

    # Tokenizing Posts and counting the length of each post
    data_frame['Tokens'] = data_frame.apply(lambda row: word_tokenize(str(row['Text'])), axis=1)
    data_frame['Length'] = data_frame.apply(lambda row: len(row['Tokens']), axis=1)

    return data_frame

# Data

### Twitter 10 000

In [ ]:
Path_Twitter_1 = "/content/drive/MyDrive/MH ML project/Datasets/Twitter/twitter-suicidal_data_10000.csv"
df = pd.read_csv(Path_Twitter_1)
df = df.rename(columns={'tweet':'Text', 'intention':'Label'})

### Twitter Tendency

In [ ]:
Path_Twitter_2 = "/content/drive/MyDrive/MH ML project/Datasets/suicidal-tendency-tweets.csv"
df = pd.read_csv(Path_Twitter_2, encoding='latin-1', usecols=['tweet', 'intention'], nrows = 17142)
df.head()

### Reddit SNS

In [ ]:
Reddit_path = "/content/drive/MyDrive/MH ML project/Datasets/Reddit_non suicide  suicide/Suicide_Detection.csv"  ## positive samples

df = pd.read_csv(Reddit_path, encoding='latin-1', usecols=['text', 'class'], nrows=20000)
df = df.rename(columns={'text': 'Text', 'class': 'Label'})

label_dict = {'suicide': 1, 'non-suicide': 0}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df['Label']

df

## Reddit _ SD

In [ ]:
## Preparing the data
reddit_path = "/content/drive/MyDrive/MH ML project/Datasets/Reddit_depressionSuicide/reddit_depression_suicidewatch.csv" 

df = pd.read_csv(reddit_path, encoding='latin-1')
df = df.rename(columns={'text':'Text', 'label':'Label'}) 
label_dict = {'depression': 0, 'SuicideWatch': 1}
df['Label'] = df['Label'].apply(lambda row: label_dict[row])
df = preprocessing(df)
df

# Main

In [ ]:
df = preprocessing(df)

# Word2Vec word embedding

In [ ]:
wd2vc_model = gensim.models.Word2Vec(df['Tokens'], min_count = 1, size = 300, window = 5)
wd2vc_model.train(df['Tokens'],epochs=10,total_examples=len(df['Tokens']))

In [ ]:
## Creating the embedding matrix
vocab = wd2vc_model.wv.vocab
print("The total number of words are : ", len(vocab))
vocab = list(vocab.keys())

word_vec_dict = {}
for word in vocab:
    word_vec_dict[word] = wd2vc_model.wv.get_vector(word)
print("The no of key-value pairs : ", len(word_vec_dict))

max_length = max(df['Length'])
print("maximum length = ", max_length)

tok = Tokenizer()
tok.fit_on_texts(df['Tokens'])
vocab_size = len(tok.word_index) + 1
encd_rev = tok.texts_to_sequences(df['Tokens'])

embed_dim = 300
pad_rev = pad_sequences(encd_rev, maxlen=max_length, padding='post')
print("pad_shape = ", pad_rev.shape)

# Rreating the embedding matrix
embed_matrix = np.zeros(shape=(vocab_size, embed_dim))
print(embed_matrix.shape)
for word, i in tok.word_index.items():
    embed_vector = word_vec_dict.get(word)
    if embed_vector is not None:
        embed_matrix[i] = embed_vector

In [ ]:
np.count_nonzero(np.isnan(pad_rev))

0

In [ ]:
from sklearn import preprocessing
pad_rev = preprocessing.normalize(pad_rev)
pad_rev

In [ ]:
## Spliting the data
X_train, X_test, y_train, y_test = train_test_split(pad_rev, df['Label'], shuffle=True, test_size=0.2, random_state=42)

print("Train = ", X_train.shape, y_train.shape)
print("Test = ", X_test.shape, y_test.shape)

Train =  (16290, 9360) (16290,)
Test =  (4073, 9360) (4073,)


# Baselines

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn import metrics
def evaluate_classfier_quality(pred,y_test):
    accuracy = metrics.accuracy_score(y_test,pred)
    print("Accuracy: %1.4f" %accuracy)
    precision = metrics.precision_score(y_test,pred,average='weighted')
    print("Precision: %1.4f" % precision)
    recall = metrics.recall_score(y_test,pred,average='weighted')
    print("Recall: %1.4f" % recall)
    f1 = metrics.f1_score(y_test,pred,average='weighted')
    print("F1: %1.4f" % f1)
    return accuracy,precision,recall,f1

### KMeans

In [ ]:
from sklearn.cluster import KMeans
##Elbow method
sse = []
num_of_k = list(range(2, 6))

kmeans = KMeans(n_clusters=2)
results = kmeans.fit(X_train)
history = kmeans.predict(X_test)
y_pred= history
evaluate_classfier_quality(y_pred,y_test)

### Random Forest

In [ ]:
model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)
evaluate_classfier_quality(y_pred,y_test)

### Decision Tree

In [ ]:
grid_param={'min_samples_split' : [10,50,100],'max_depth': [2,4,8]}
model_dt = GridSearchCV(DecisionTreeClassifier(),param_grid=grid_param,cv=5,n_jobs=-1)
model_dt.fit(X_train,y_train)
y_pred = model_dt.predict(X_test)
evaluate_classfier_quality(y_pred,y_test)

### Gradient Boost

In [ ]:
model_gb = GradientBoostingClassifier()
model_gb.fit(X_train,y_train)
y_pred = model_gb.predict(X_test)
evaluate_classfier_quality(y_pred,y_test)


### SVM

In [ ]:
from sklearn.svm import SVC
model_svm = SVC(kernel='linear')
model_svm.fit(X_train, y_train)
y_pred = model_svm.predict(X_test)
evaluate_classfier_quality(y_pred,y_test)

### Logistic Regression

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
evaluate_classfier_quality(y_pred,y_test)

### LSTM-CNN

In [ ]:
X_train_encoded = X_train
X_test_encoded = X_test
X_val_encoded = X_val

In [ ]:
X_train_encoded_reshaped = X_train_encoded.reshape(-1, 1, X_train_encoded.shape[1])
X_test_encoded_reshaped  = X_test_encoded.reshape(-1, 1, X_test_encoded.shape[1])
X_val_encoded_reshaped = X_val_encoded.reshape(-1, 1, X_val_encoded.shape[1])

In [ ]:
y_train_reshaped = y_train.values.reshape(-1, 1)
y_test_reshaped = y_test.values.reshape(-1, 1)
y_val_reshaped = y_val.values.reshape(-1, 1)

In [ ]:
print("Train = ", X_train_encoded_reshaped.shape, y_train_reshaped.shape)
print("Test = ", X_test_encoded_reshaped.shape, y_test_reshaped.shape)
print("Validation = ", X_val_encoded_reshaped.shape, y_val_reshaped.shape)

Train =  (17136, 1, 134) (17136, 1)
Test =  (5356, 1, 134) (5356, 1)
Validation =  (4284, 1, 134) (4284, 1)


In [ ]:
from keras.layers.activation.leaky_relu import LeakyReLU

lstm_cnn_model = tf.keras.models.Sequential()
lstm_cnn_model.add(tf.keras.layers.Input(shape=(1, X_train_encoded_reshaped.shape[2])))
lstm_cnn_model.add(tf.keras.layers.Dropout(0.5))
lstm_cnn_model.add(tf.keras.layers.LSTM(units=100, input_shape=(1,64), return_sequences = True))
lstm_cnn_model.add(tf.keras.layers.Conv1D(3, (8,), padding='same', activation='relu'))
lstm_cnn_model.add(tf.keras.layers.Flatten())
lstm_cnn_model.add(Dense(1, activation='sigmoid')) ##4labels

lstm_cnn_model.summary()

In [ ]:
## Compiling and fitting
lstm_cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# specify batch size and epochs for training.
epochs = 10
batch_size = 32
r = lstm_cnn_model.fit(X_train_encoded_reshaped, y_train_reshaped, 
              epochs=epochs, batch_size=batch_size, 
              validation_data=(X_val_encoded_reshaped, y_val_reshaped))

In [ ]:
#Prediction on the test data
pred = lstm_cnn_model.predict(X_test_encoded_reshaped)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
    if pred[item] >= 0.5:
      y_pred[item] = 1
    else:
      y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)

### CNN

In [ ]:
from keras.layers.activation.leaky_relu import LeakyReLU
from keras.initializers import Constant

cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.layers.Input(shape=(1, X_train_encoded_reshaped.shape[2])))
cnn_model.add(tf.keras.layers.Dropout(0.5))
cnn_model.add(tf.keras.layers.Conv1D(3, (8,), padding='same'))
cnn_model.add(tf.keras.layers.LeakyReLU(0.05))

cnn_model.add(tf.keras.layers.Flatten())
cnn_model.add(Dense(1, activation='sigmoid')) ##4labels

cnn_model.summary()

In [ ]:
## Compiling and fitting
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# specify batch size and epochs for training.
epochs = 10
batch_size = 32
r = cnn_model.fit(X_train_encoded_reshaped, y_train_reshaped, 
              epochs=epochs, batch_size=batch_size, 
              validation_data=(X_val_encoded_reshaped, y_val_reshaped))

In [ ]:
#Prediction on the test data
pred = cnn_model.predict(X_test_encoded_reshaped)

In [ ]:
len(y_test_reshaped)
y_test = y_test_reshaped

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
    # y_pred[item] = list(pred[item]).index(max(pred[item]))
    if pred[item] >= 0.5:
      y_pred[item] = 1
    else:
      y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)

### LSTM

In [ ]:
from keras.layers.activation.leaky_relu import LeakyReLU
from keras.initializers import Constant

lstm_model = tf.keras.models.Sequential()
lstm_model.add(tf.keras.layers.Input(shape=(1, X_train_encoded_reshaped.shape[2])))
lstm_model.add(tf.keras.layers.Dropout(0.5))
lstm_model.add(tf.keras.layers.LSTM(units=100, input_shape=(1,64), return_sequences = True))
lstm_model.add(tf.keras.layers.Flatten())
lstm_model.add(Dense(1, activation='sigmoid')) ##4labels

lstm_model.summary()

In [ ]:
## Compiling and fitting
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# specify batch size and epochs for training.
epochs = 10
batch_size = 32
r = lstm_model.fit(X_train_encoded_reshaped, y_train_reshaped, 
              epochs=epochs, batch_size=batch_size, 
              validation_data=(X_val_encoded_reshaped, y_val_reshaped))

In [ ]:
#Prediction on the test data
pred = lstm_model.predict(X_test_encoded_reshaped)
print(pred)

In [ ]:
y_pred = np.zeros(len(pred))
for item in range(len(pred)):
    # y_pred[item] = list(pred[item]).index(max(pred[item]))
    if pred[item] >= 0.5:
      y_pred[item] = 1
    else:
      y_pred[item] = 0

print(y_pred)

In [ ]:
import sklearn

report = sklearn.metrics.classification_report(y_test, y_pred)
print(report)
accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
precision = sklearn.metrics.precision_score(y_test, y_pred)
recall = sklearn.metrics.recall_score(y_test, y_pred)
f1score = sklearn.metrics.f1_score(y_test, y_pred)
print("-----------------------")
print(accuracy)
print(precision)
print(recall)
print(f1score)